In [2]:
from dotenv import load_dotenv
from googleapiclient.discovery import build
import os
load_dotenv()
import pandas as pd
API_KEY = os.getenv('API_KEY')

In [3]:
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [4]:
channel_ids  = ['UCvPGx8qh8AkrFZW06tESvaA', #DotDager
    'UC4FHiPgS1KXkUMx3dxBUtPg', #HolaMundo
                # 'UCjXAQ-cayM4mIZmUZKMFW_w', #Programador X
                ]

# Obtener id lista de reproducción videos.

In [5]:
channel_ids

['UCvPGx8qh8AkrFZW06tESvaA', 'UC4FHiPgS1KXkUMx3dxBUtPg']

In [6]:
request = youtube.channels().list(
    part='snippet, contentDetails, statistics',
    id=channel_ids
)
response = request.execute()

In [7]:
info_channels = []
for item in response.get('items', []):
    channel_info = {
        'title': item['snippet']['title'],
        'country': item['snippet']['country'],
        'subscriberCount': item['statistics']['subscriberCount'],
        'id': item['id'],
        'uploads': item['contentDetails']['relatedPlaylists']['uploads'],
        'viewCount': item['statistics']['viewCount'],
        'subscriberCount': item['statistics']['subscriberCount'],
        'videoCount': item['statistics']['videoCount'],
    }
    info_channels.append(channel_info)
df = pd.DataFrame(info_channels)
df

,title,country,subscriberCount,id,uploads,viewCount,videoCount
0,Dot Dager,AR,52200,UCvPGx8qh8AkrFZW06tESvaA,UUvPGx8qh8AkrFZW06tESvaA,1424225,41
1,HolaMundo,NZ,768000,UC4FHiPgS1KXkUMx3dxBUtPg,UU4FHiPgS1KXkUMx3dxBUtPg,44916689,368


# Obtener id de videos.

In [8]:
request = youtube.playlistItems().list(
    part="contentDetails",
    playlistId='UUvPGx8qh8AkrFZW06tESvaA'
)
response = request.execute()
response

{'kind': 'youtube#playlistItemListResponse',
 'etag': 'spGvEzLAo9C5NDQWDOJucNg-32M',
 'nextPageToken': 'EAAafVBUOkNBVWlFRGM0UkRVNU9UZzROVFkzUVRsRlJqTW9BVWl6MnFhanFQV0ZBMUFCV2pnaVEyaG9WbFpZV2xGU00yYzBZMWRuTkZGWGRIbFNiSEJZVFVSYU1GSldUakpaVlVWVFJFRnFVblJrZFhoQ2FFTTBlWEZoTjBGbkln',
 'items': [{'kind': 'youtube#playlistItem',
   'etag': 'n3BsKMqm6c_x6EFG0eAtYUOCQHw',
   'id': 'VVV2UEd4OHFoOEFrckZaVzA2dEVTdmFBLkY5c0xrM0VkTFA0',
   'contentDetails': {'videoId': 'F9sLk3EdLP4',
    'videoPublishedAt': '2024-05-05T01:03:12Z'}},
  {'kind': 'youtube#playlistItem',
   'etag': 'hSvi6zdA59lb_4bongLLGyhKQFA',
   'id': 'VVV2UEd4OHFoOEFrckZaVzA2dEVTdmFBLkhMQzBZWFZjNW1B',
   'contentDetails': {'videoId': 'HLC0YXVc5mA',
    'videoPublishedAt': '2024-05-05T00:50:00Z'}},
  {'kind': 'youtube#playlistItem',
   'etag': 'xv6Tol0v5l4Scb8qHOhFC-Z_FdA',
   'id': 'VVV2UEd4OHFoOEFrckZaVzA2dEVTdmFBLmNuY0k5XzNJbzJj',
   'contentDetails': {'videoId': 'cncI9_3Io2c',
    'videoPublishedAt': '2024-04-28T00:38:14Z'}},
  {'

In [9]:
videoId_list = []

while True:
    # Procesar los elementos de la página actual
    for item in response['items']:
        videoId_list.append(item['contentDetails']['videoId'])

    # Verificar si hay más páginas
    if 'nextPageToken' in response:
        # Si hay más páginas, solicitar la siguiente página
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId='UUvPGx8qh8AkrFZW06tESvaA',
            pageToken=response['nextPageToken']
        )
        response = request.execute()
    else:
        # Si no hay más páginas, salir del bucle
        break

# Ahora 'videoId_list' contiene todos los IDs de los videos de la lista de reproducción


In [10]:
len(videoId_list)

42

# Obtener información de videos.

In [28]:
data = []

for videoId in videoId_list:
    next_page_token = None
    while True:
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=videoId,
            pageToken=next_page_token
        )
        response = request.execute()
        items = response.get('items', [])
        for item in items:
            data.append({
                'videoId': item['id'],
                'publishedAt': item['snippet']['publishedAt'],
                'title': item['snippet']['title'],
                'description': item['snippet']['description'],
                'channelTitle': item['snippet']['channelTitle'],
                'tags': item['snippet']['tags'],
                'categoryId': item['snippet']['categoryId'],
                'liveBroadcastContent': item['snippet']['liveBroadcastContent'],
                'xd': item['snippet'].get('defaultLanguage'),
                'duration': item['contentDetails']['duration'],
                'dimension': item['contentDetails']['dimension'],
                'definition': item['contentDetails']['definition'],
                'viewCount': item['statistics']['viewCount'],
                'likeCount': item['statistics']['likeCount'],
                'commentCount': item['statistics']['commentCount'],
            })
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

print(data)


[{'videoId': 'F9sLk3EdLP4', 'publishedAt': '2024-05-05T01:03:12Z', 'title': 'Falopa y programando un server DNS en C#  (parte 4)', 'description': 'Usamos CodeCrafters para hacer un server DNS con CSharp.\nSi te copa la plataforma podés usar este link para unirte: https://app.codecrafters.io/join?via=MarianoVilla\nSi no quieren poner la plata -entendible-, pueden hacer los proyectos sin modo interactivo con este repo: https://github.com/codecrafters-io/build-your-own-x\n\nAlgunos recursos copados para ahondar en DNS:\n\nEl RFC base:\nhttps://datatracker.ietf.org/doc/html/rfc1035\nIntro a DNS:\nhttps://blog.bytebytego.com/p/a-crash-course-in-dns-domain-name\nGuía más profunda de DNS:\nhttps://github.com/EmilHernvall/dnsguide/blob/b52da3b32b27c81e5c6729ac14fe01fef8b1b593/chapter1.md\nUn sitio muy interesante para buscar info:\ngoogle.com\n•·················•·················•\r\n\r\n00:00 Acá empieza el stream\n05:15 Q&A: dónde aprender Git, cursos, lógica\n31:40 Lunfardo: un lenguaje de 

In [32]:
df2 = pd.DataFrame(data)
df2.head()

,videoId,publishedAt,title,description,channelTitle,tags,categoryId,liveBroadcastContent,xd,duration,dimension,definition,viewCount,likeCount,commentCount
0,F9sLk3EdLP4,2024-05-05T01:03:12Z,Falopa y programando un server DNS en C# (par...,Usamos CodeCrafters para hacer un server DNS c...,Dot Dager,"[programación, aprender a programar, programad...",24,none,es-419,PT3H47M44S,2d,hd,3501,218,3
1,HLC0YXVc5mA,2024-05-05T00:50:00Z,¡Falopa! Ahora que tengo tu atención... ¡Falopa!,Esto es una descripción.\n\n•·················...,Dot Dager,"[programación, aprender a programar, programad...",24,upcoming,es-419,P0D,2d,sd,0,0,0
2,cncI9_3Io2c,2024-04-28T00:38:14Z,"Falopa, Q&A y... eso.",Esto es una descripción.\n\n•·················...,Dot Dager,"[programación, aprender a programar, programad...",24,none,es-419,PT3H17M55S,2d,hd,4684,414,11
3,NxMpm10Ybd4,2024-04-22T01:36:54Z,Vamo' a (charlar y) programar un server DNS en...,Usamos CodeCrafters para hacer un server DNS c...,Dot Dager,"[programación, aprender a programar, programad...",24,none,es-419,PT4H19M25S,2d,hd,5106,331,3
4,IZAYA37KUHk,2024-04-17T17:35:06Z,C# no es un lenguaje de backend.,"Bueno, sí, CSharp es un lenguaje de back-end, ...",Dot Dager,"[programación, aprender a programar, programad...",24,none,es-419,PT12M40S,2d,hd,26579,2817,246


In [ ]:
# playlist_id = ''
# print(f'https://www.youtube.com/playlist?list={playlist_id}')
